In [1]:
!nvidia-smi

Wed May 29 17:52:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git


Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25
Receiving objects: 100% (39/39), 177.08 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (13/13), done.


In [3]:
#directory
data_dir = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [4]:
# Check all of the filenames in the target directory
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 'pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 'pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

#preprocessing the  data

In [5]:
def get_lines(filename):
  """
  Reads filename (a text file) and returns the lines of text as a list.

  Args:
      filename: a string containing the target filepath to read.

  Returns:
      A list of strings with one string per line from the target filename.
      For example:
      ["this is the first line of filename",
       "this is the second line of filename",
       "..."]
  """
  with open(filename, "r") as f:
    return f.readlines()

In [6]:
train_lines = get_lines(data_dir + "train.txt")
train_lines[:20]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

## get the lines on the test data

In [7]:
test_lines = get_lines(data_dir + "test.txt")
test_lines[:20]

['###24845963\n',
 'BACKGROUND\tThis study analyzed liver function abnormalities in heart failure patients admitted with severe acute decompensated heart failure ( ADHF ) .\n',
 'RESULTS\tA post hoc analysis was conducted with the use of data from the Evaluation Study of Congestive Heart Failure and Pulmonary Artery Catheterization Effectiveness ( ESCAPE ) .\n',
 'RESULTS\tLiver function tests ( LFTs ) were measured at @ time points from baseline , at discharge , and up to @ months follow-up .\n',
 'RESULTS\tSurvival analyses were used to assess the association between admission Model of End-Stage Liver Disease Excluding International Normalized Ratio ( MELD-XI ) scores and patient outcome.There was a high prevalence of abnormal baseline ( admission ) LFTs ( albumin @ % , aspartate transaminase @ % , alanine transaminase @ % , and total bilirubin @ % ) .\n',
 "RESULTS\tThe percentage of patients with abnormal LFTs decreased significantly from baseline to @-months ' follow-up .\n",
 'RE

Arrange the data in this format
```
[
  {'line_number': 0,
  'target': 'OBJECTIVE',
  'text': 'to investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( oa ) .',
  'total_lines': 11},
  ...]
```

In [8]:
def preprocess_text_with_line_numbers(input_lines):
  abstract_lines = "" # create an empty abstract
  abstract_samples = [] # create an empty list of abstracts

  # Loop through each line in target file
  for line in input_lines:
    if line.startswith("###"): # check to see if line is an ID line
      abstract_id = line
      abstract_lines = "" # reset abstract string
    elif line.isspace(): # check to see if line is a new line
      abstract_line_split = abstract_lines.splitlines() # split abstract into separate lines

      # Iterate through each line in abstract and count them at the same time
      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {} # create empty dict to store data from line
        target_text_split = abstract_line.split("\t") # split target label from text
        line_data["target"] = target_text_split[0] # get target label
        line_data["text"] = target_text_split[1].lower() # get target text and lower it
        line_data["line_number"] = abstract_line_number # what number line does the line appear in the abstract?
        line_data["total_lines"] = len(abstract_line_split) - 1 # how many total lines are in the abstract? (start from 0)
        abstract_samples.append(line_data) # add line data to abstract samples list

    else: # if the above conditions aren't fulfilled, the line contains a labelled sentence
      abstract_lines += line

  return abstract_samples

In [9]:
validate_lines = get_lines(data_dir + "dev.txt" )
train_sample = preprocess_text_with_line_numbers(train_lines)
test_sample = preprocess_text_with_line_numbers(test_lines)
validation_sample = preprocess_text_with_line_numbers(validate_lines)
train_sample[:10]


[{'target': 'OBJECTIVE',
  'text': 'to investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( oa ) .',
  'line_number': 0,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'a total of @ patients with primary knee oa were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .',
  'line_number': 1,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .',
  'line_number': 2,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'pain was assessed using the visual analog pain scale ( @-@ mm ) .',
  'line_number': 3,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'secondary outcome measures included the western ontari

## Change the data to dataframe for a better visualization

In [10]:
import pandas as pd

train_df = pd.DataFrame(train_sample)
test_df = pd.DataFrame(test_sample)
validate_df = pd.DataFrame(validation_sample)
train_df.head(15)

,target,text,line_number,total_lines
0,OBJECTIVE,to investigate the efficacy of @ weeks of dail...,0,11
1,METHODS,a total of @ patients with primary knee oa wer...,1,11
2,METHODS,outcome measures included pain reduction and i...,2,11
3,METHODS,pain was assessed using the visual analog pain...,3,11
4,METHODS,secondary outcome measures included the wester...,4,11
5,METHODS,"serum levels of interleukin @ ( il-@ ) , il-@ ...",5,11
6,RESULTS,there was a clinically relevant reduction in t...,6,11
7,RESULTS,the mean difference between treatment arms ( @...,7,11
8,RESULTS,"further , there was a clinically relevant redu...",8,11
9,RESULTS,these differences remained significant at @ we...,9,11


In [11]:
train_sentences = train_df["text"].tolist()
val_sentences = validate_df["text"].tolist()
test_sentences = test_df["text"].tolist()

## Make numeric labels (ML models require numeric labels)


In [12]:
# One hot encode labels
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False)
train_labels_one_hot = one_hot_encoder.fit_transform(train_df["target"].to_numpy().reshape(-1, 1))
val_labels_one_hot = one_hot_encoder.transform(validate_df["target"].to_numpy().reshape(-1, 1))
test_labels_one_hot = one_hot_encoder.transform(test_df["target"].to_numpy().reshape(-1, 1))

# Check what training labels look like
train_labels_one_hot

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

### Label encode labels

In [13]:
# Extract labels ("target" columns) and encode them into integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_df["target"].to_numpy())
val_labels_encoded = label_encoder.transform(validate_df["target"].to_numpy())
test_labels_encoded = label_encoder.transform(test_df["target"].to_numpy())

# Check what training labels look like
train_labels_encoded

array([3, 2, 2, ..., 4, 1, 1])

In [14]:
# Get class names from LabelEncoder instance
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_
class_names

array(['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS'],
      dtype=object)

# Model 1 Naive Bayes Classifier

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create a pipeline
model_0 = Pipeline([
  ("tf-idf", TfidfVectorizer()),
  ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(X=train_sentences,
            y=train_labels_encoded);

In [16]:
# Evaluate baseline on validation dataset
model_0.score(X=val_sentences,
              y=val_labels_encoded)

0.7218323844829869

In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):

  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [18]:
validate_pred = model_0.predict(val_sentences)
calculate_results(val_labels_encoded , validate_pred)

{'accuracy': 72.1832384482987,
 'precision': 0.7186466952323352,
 'recall': 0.7218323844829869,
 'f1': 0.6989250353450294}

## Preparing our data for deep sequence models


In [19]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers

In [20]:
max_tokens = 68000
# Create text vectorizer

from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=max_tokens,
                                    output_sequence_length=55)

text_vectorizer.adapt(train_sentences)


In [21]:
# Test out text vectorizer
import random
target_sentence = random.choice(train_sentences)
print(f"Text:\n{target_sentence}")
print(f"\nLength of text: {len(target_sentence.split())}")
print(f"\nVectorized text:\n{text_vectorizer([target_sentence])}")

Text:
the measurements were performed @ times : at baseline ( t@ ) , as well as @ weeks ( t@ ) , @ months ( t@ ) , and @ months after baseline ( t@ ) .

Length of text: 37

Vectorized text:
[[  2 591   9 153 361  15  49 206  25 221  25  53 206  41 206   3  41  21
   49 206   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]]


In [22]:
# How many words in our training vocabulary?
rct_20k_text_vocab = text_vectorizer.get_vocabulary()

In [23]:
# Create token embedding layer
token_embed = layers.Embedding(input_dim=len(rct_20k_text_vocab), # length of vocabulary
                               output_dim=128,
                               mask_zero=True,
                               name="token_embedding")

In [24]:
# Turn our data into TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels_one_hot))
valid_dataset = tf.data.Dataset.from_tensor_slices((val_sentences, val_labels_one_hot))
test_dataset = tf.data.Dataset.from_tensor_slices((test_sentences, test_labels_one_hot))

train_dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [25]:
# Take the TensorSliceDataset's and turn them into prefetched batches
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 5), dtype=tf.float64, name=None))>

## Model 2 Deep learning

In [32]:
# Create 1D convolutional model to process sequences
inputs = layers.Input(shape=(1,), dtype=tf.string)
text_vectors = text_vectorizer(inputs) # vectorize text inputs
token_embeddings = token_embed(text_vectors) # create embedding
x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(token_embeddings)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model_1 = tf.keras.Model(inputs, outputs)

# Compile
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_1_history = model_1.fit(train_dataset,
                              steps_per_epoch=int(0.1 * len(train_dataset)), # only fit on 10% of batches for faster training time
                              epochs=3,
                              validation_data=valid_dataset,
                              validation_steps=int(0.1 * len(valid_dataset))) # only validate on 10% of batches

Epoch 1/3
562/562 [==============================] - 22s 38ms/step - loss: 0.5431 - accuracy: 0.8205 - val_loss: 0.5783 - val_accuracy: 0.8032
Epoch 2/3
562/562 [==============================] - 5s 9ms/step - loss: 0.3716 - accuracy: 0.8751 - val_loss: 0.5921 - val_accuracy: 0.7912
Epoch 3/3
562/562 [==============================] - 5s 9ms/step - loss: 0.3475 - accuracy: 0.8848 - val_loss: 0.6035 - val_accuracy: 0.7882


In [34]:
# from tensorflow.keras.callbacks import  EarlyStopping,TensorBoard  # Assuming TensorFlow for callbacks

# # Define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=3)  # Stop training if validation loss doesn't improve for 3 epochs
# tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)  # TensorBoard visualization (adjust log directory)

# Fit the model with callbacks
model_1_history = model_1.fit(train_dataset,
        steps_per_epoch=int(0.1 * len(train_dataset)),
        epochs=10,
        validation_data=valid_dataset,
        validation_steps=int(0.1 * len(valid_dataset)),
        # callbacks=[early_stopping, tensorboard])
)


Epoch 1/10
562/562 [==============================] - 4s 7ms/step - loss: 0.3034 - accuracy: 0.8989 - val_loss: 0.6265 - val_accuracy: 0.7999
Epoch 2/10
562/562 [==============================] - 4s 7ms/step - loss: 0.2613 - accuracy: 0.9163 - val_loss: 0.6605 - val_accuracy: 0.7842
Epoch 3/10
562/562 [==============================] - 4s 7ms/step - loss: 0.2461 - accuracy: 0.9229 - val_loss: 0.6943 - val_accuracy: 0.7733
Epoch 4/10
562/562 [==============================] - 4s 8ms/step - loss: 0.4136 - accuracy: 0.8583 - val_loss: 0.6090 - val_accuracy: 0.7906
Epoch 5/10
562/562 [==============================] - 5s 9ms/step - loss: 0.4162 - accuracy: 0.8579 - val_loss: 0.5740 - val_accuracy: 0.7995
Epoch 6/10
562/562 [==============================] - 4s 8ms/step - loss: 0.4072 - accuracy: 0.8575 - val_loss: 0.5697 - val_accuracy: 0.7952
Epoch 7/10
562/562 [==============================] - 5s 8ms/step - loss: 0.3927 - accuracy: 0.8655 - val_loss: 0.5590 - val_accuracy: 0.8039
Epoch 

In [36]:
model_1.evaluate(valid_dataset)

945/945 [==============================] - 6s 6ms/step - loss: 0.5249 - accuracy: 0.8117


[0.5249070525169373, 0.8117304444313049]

In [37]:
# Make predictions (our model outputs prediction probabilities for each class)
model_1_pred_probs = model_1.predict(valid_dataset)
model_1_pred_probs

945/945 [==============================] - 2s 2ms/step


array([[6.4993018e-01, 3.6786854e-02, 1.7506681e-01, 8.9334182e-02,
        4.8881955e-02],
       [5.6009960e-01, 1.9349538e-01, 1.6786981e-02, 2.0126465e-01,
        2.8353367e-02],
       [1.4778842e-01, 3.4631009e-03, 1.7659588e-03, 8.4689999e-01,
        8.2457351e-05],
       ...,
       [1.5561692e-06, 3.3350578e-05, 2.8475244e-03, 2.0382183e-06,
        9.9711549e-01],
       [5.8952693e-02, 5.4739994e-01, 1.4198275e-01, 4.5019969e-02,
        2.0664464e-01],
       [4.4815666e-03, 9.9076700e-01, 3.8331135e-03, 9.3906143e-05,
        8.2457397e-04]], dtype=float32)

In [39]:
# Convert pred probs to classes
model_1_preds = tf.argmax(model_1_pred_probs, axis=1)

calculate_results(val_labels_encoded , model_1_preds)

{'accuracy': 81.17304382364622,
 'precision': 0.8078765154514085,
 'recall': 0.8117304382364623,
 'f1': 0.808903846999711}

In [41]:
# Fit the model with callbacks
model_1_history = model_1.fit(train_dataset,
        steps_per_epoch=int(0.1 * len(train_dataset)),
        epochs=11,
        validation_data=valid_dataset,
        validation_steps=int(0.1 * len(valid_dataset)),
)


Epoch 1/11
562/562 [==============================] - 4s 7ms/step - loss: 0.1892 - accuracy: 0.9433 - val_loss: 0.6494 - val_accuracy: 0.7985
Epoch 2/11
562/562 [==============================] - 3s 6ms/step - loss: 0.2598 - accuracy: 0.9128 - val_loss: 0.6620 - val_accuracy: 0.7839
Epoch 3/11
562/562 [==============================] - 3s 6ms/step - loss: 0.2369 - accuracy: 0.9237 - val_loss: 0.6945 - val_accuracy: 0.7756
Epoch 4/11
562/562 [==============================] - 5s 8ms/step - loss: 0.3210 - accuracy: 0.8947 - val_loss: 0.6463 - val_accuracy: 0.7892
Epoch 5/11
562/562 [==============================] - 3s 6ms/step - loss: 0.3369 - accuracy: 0.8889 - val_loss: 0.6114 - val_accuracy: 0.7959
Epoch 6/11
562/562 [==============================] - 3s 6ms/step - loss: 0.3371 - accuracy: 0.8876 - val_loss: 0.6059 - val_accuracy: 0.7919
Epoch 7/11
562/562 [==============================] - 4s 6ms/step - loss: 0.3267 - accuracy: 0.8917 - val_loss: 0.5921 - val_accuracy: 0.7995
Epoch 

562/562 [==============================] - 0s 686us/step - loss: 0.6277 - accuracy: 0.8100 - val_loss: 0.5739 - val_accuracy: 0.7929
